In [1]:
import sys
sys.path.append("..")
from code_pytorch import *
from nice.blocks import *
from nice.utilities import *
import numpy as np
import ase.io
import copy
import os

In [2]:
if ('methane.extxyz' not in os.listdir('.')):
        os.system('wget "https://archive.materialscloud.org/record/file?file_id=b612d8e3-58af-4374-96ba-b3551ac5d2f4&filename=methane.extxyz.gz&record_id=528" -O methane.extxyz.gz')
        os.system("gunzip -k methane.extxyz.gz")

structures = ase.io.read('methane.extxyz', index='0:20')


In [66]:
def get_invariance(structures):
    N_MAX = 5
    L_MAX = 5
    HYPERS = {
        'interaction_cutoff': 6.3,
        'max_radial': N_MAX,
        'max_angular': L_MAX,
        'gaussian_sigma_type': 'Constant',
        'gaussian_sigma_constant': 0.3,
        'cutoff_smooth_width': 0.3,
        'radial_basis': 'GTO'
    }
    
    all_species = get_all_species(structures)

    coefficients = get_spherical_expansion(structures, HYPERS,
                                                 all_species, show_progress = False)
    
    for key in coefficients.keys():
        print(key, coefficients[key].shape)
        
    variances = {}
    for key in coefficients.keys():
        variances[key] = {}
        for l in range(coefficients[key].shape[2]):
            now = coefficients[key][:, :, l, :2*l + 1]
            #print(now.shape)
            variances[key][l] = 0.0
            for m in range(-l, l + 1):
                #print( np.sum(now[:, :, m + l] * now[:, :, m + l], axis =1).shape)
                if (m != 0):
                    variances[key][l] += np.mean(np.sum(now[:, :, m + l] * now[:, :, m + l], axis =1))
                else:
                    variances[key][l] += 1.0 * np.mean(np.sum(now[:, :, m + l] * now[:, :, m + l], axis =1))
            
    return variances

In [67]:
variances_before = get_invariance(structures)

1 (80, 10, 6, 11)
6 (20, 10, 6, 11)


In [68]:
rotated_structures = copy.deepcopy(structures)
for structure in rotated_structures:
    structure.euler_rotate(np.random.rand() * 360, np.random.rand() * 360, np.random.rand() * 360)

In [69]:
variances_after = get_invariance(rotated_structures)

1 (80, 10, 6, 11)
6 (20, 10, 6, 11)


In [70]:
for key in variances_before.keys():
    for l in variances_before[key].keys():
        print(variances_before[key][l], variances_after[key][l],
              np.abs(variances_before[key][l] - variances_after[key][l]))

0.0001656811834181743 0.00016568118341817438 8.131516293641283e-20
5.0135158404901264e-05 5.0135158404901264e-05 0.0
5.993970375184746e-05 5.993970375184744e-05 1.3552527156068805e-20
6.76152892840363e-05 6.76152892840363e-05 0.0
7.141161961740204e-05 7.14116196174021e-05 6.776263578034403e-20
6.803614762335905e-05 6.803614762335912e-05 6.776263578034403e-20
0.0001953002788311111 0.0001953002788311111 0.0
4.7752706338244424e-05 4.775270633824446e-05 3.3881317890172014e-20
8.034560249778188e-05 8.034560249778195e-05 6.776263578034403e-20
9.975744907905935e-05 9.975744907905945e-05 9.486769009248164e-20
9.984613537464386e-05 9.984613537464389e-05 2.710505431213761e-20
8.334681229293096e-05 8.334681229293098e-05 2.710505431213761e-20
